In [2]:
import json
import math
from collections import Counter # to count and sort labels used in DBScan algo
from statistics import StatisticsError # to catch stdv errors with sotherms that hold only 1 data point 
import statistics
import requests
import pandas as pd
import pprint
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.cluster import DBSCAN # for methods in DBScan

isothermLib = requests.get("https://adsorption.nist.gov/isodb/api/isotherms.json").json()
total = 0
hcount = 0
noncount = 0
differences = list()

def dbScan(yv, xv, df, diff):
  "cluster detection" 

  avg_eps = math.floor((statistics.mean(diff))) # finds average epsilon value depending on avg distance between each point from its neighbor
  dataf = df.iloc[:,0:]
  try:
    model = DBSCAN(eps=avg_eps, min_samples=3).fit(dataf)
  except ValueError:
    model = DBSCAN(eps=1, min_samples=3).fit(dataf)

  outliers_df = pd.DataFrame(dataf)

  # for printing dbscan model

  #print(model)
  #print(Counter(model.labels_))
  #print(outliers_df[model.labels_==-1])

  # setting up the scatter plot
  #fig = plt.figure()
  #ax = fig.add_axes([.1, .1, 1, 1])
  #colors = model.labels_
  #scatter = ax.scatter(dataf['pressure'], dataf['adsorption'], c=colors, s=120)
  #ax.set_xlabel('pressure')
  #ax.set_ylabel('adsorption')
  #plt.title('DBScan')
  #legend1 = ax.legend(*scatter.legend_elements(),
  #                 loc="upper left", title="Labels")
  #plt.show()


  ob = Counter(model.labels_)

  # filtering out 2 cluters
  if (len(ob)) == 2:
    if -1 not in ob.keys():
      return True

  return False

def hysteresis(x, y, filename, df):
  "determines hysteresis"

  # we will assume that isotherms with less than 3 data point are not hysteresis 
  try:
    if len(y) < 3:
      return False
    else:
      stdy = statistics.stdev(y)
  except StatisticsError:
      return False 

  yvals = list()
  spike = 0

  # compares pairs of y values to see if there are big spikes between a pair
  for yval in range(len(y)-1):
    yvals.append(y[yval])
    yvals.append(y[yval+1])
    if ((statistics.stdev(yvals) < min(yvals)) or (statistics.stdev(yvals) > max(yvals))):
      spike += 1
      differences.append(abs(yvals[1] - yvals[0]))
      yvals.clear()
    else:
      yvals.clear()

  # check for clustering
  if spike > statistics.mean(y):
    if dbScan(y, x, df, differences) is True:
      return True
  else:
    return False
   
def isotherms(fname):
  "Filters multicomponent species and returns isotherm"

  pressure = []
  adsorption = []
  
  for isotherm in isothermLib:
    if fname in isotherm["filename"]:
      filename = isotherm["filename"]
      isoIsFull = requests.get(f'https://adsorption.nist.gov/isodb/api/isotherm/{filename}.json').json()
      try:
        if isotherm["adsorbates"][1]:
          return False
      except IndexError:
        #collecting x, y data
        counter = 0
        for key in isoIsFull["isotherm_data"]:
          pressure.append(isoIsFull["isotherm_data"][counter]["pressure"]) 
          adsorption.append(isoIsFull["isotherm_data"][counter]["species_data"][0]["adsorption"]) 
          counter += 1

        # creates a pandas dataframe to better hold the data for dbscan
        lib = requests.get(f'https://adsorption.nist.gov/isodb/api/isotherm/{filename}.json')
        x = lib.json()
        data = json.dumps(x['filename'])
        data = json.loads(json.dumps(x['filename']))
        df = pd.DataFrame()
        df['pressure'] = pressure
        df['adsorption'] = adsorption

        # displays hysteresis prompts
        if hysteresis(pressure, adsorption, filename, df) is True:
          print("\nPossibility of a hysteresis in", filename)
          print("\n")
          differences.clear()
          return True
        else:
          print("\nNo hysteresis in", filename)
          differences.clear()
          return False
  
# Function calls for testing
for iso in isothermLib[:10000]:
  if isotherms(iso["filename"]) is True:
    hcount += 1
  else:
    noncount += 1
  total += 1

# if you want to look at specific isotherms
#isotherms("10.1016j.ijhydene.2010.07.089.Isotherm3")
  
print("\n\nNumber of hysteresis isotherms detected found:", hcount)
print("Number of non hysteresis isotherms detected found:", noncount)
print("Total number of isotherms found:", total)
print("\n")

# comment these last two out if looking at specific isotherms
print("Percent of non hysteresis to total isotherms found:", "{:.2f}".format((noncount/total)*100) + "%")
print("Percent of hysteresis to total isotherms found:", "{:.2f}".format((hcount/total)*100) + "%")


No hysteresis in 10.1016j.ijhydene.2010.07.089.Isotherm7


Number of hysteresis isotherms detected found: 0
Number of non hysteresis isotherms detected found: 0
Total number of isotherms found: 0


